In [1]:
# NBVAL_IGNORE_OUTPUT
# Some jupyter notebook magic to reload modules automatically when they change
# not necessary for this specific notebook but useful in general
%load_ext autoreload
%autoreload 2

In [2]:
# NBVAL_IGNORE_OUTPUT
from definitions import ROOT_DIR
import os

# The PSDM specific input models can be imported from the pypsdm.models.input and
# pypsdm.models.result. The `GridWithResults` container is located in pypsdm.models.gwr
from pypsdm.models.gwr import GridWithResults

grid_path = os.path.join(ROOT_DIR, "tests", "resources", "simbench", "input")
result_path = os.path.join(ROOT_DIR, "tests", "resources", "simbench", "results")
# IO data models in general have a from_csv method to parse psdm files
gwr = GridWithResults.from_csv(grid_path, result_path)

2025-08-20 11:36:45.491 | INFO     | pypsdm.models.gwr:from_csv:293 - Reading grid from /home/smdafeis/github/pypsdm/tests/resources/simbench/input
2025-08-20 11:36:45.895 | DEBUG    | pypsdm.models.primary_data:from_csv:273 - No primary data in path /home/smdafeis/github/pypsdm/tests/resources/simbench/input
2025-08-20 11:36:45.896 | INFO     | pypsdm.models.gwr:from_csv:305 - Reading results from /home/smdafeis/github/pypsdm/tests/resources/simbench/results
2025-08-20 11:36:46.548 | WARNING  | pypsdm.models.result.participant.dict:from_csv:76 - Entity 557b9f51-d83c-476c-a84c-d240530c203d not in input entities
2025-08-20 11:36:46.552 | WARNING  | pypsdm.models.result.participant.dict:from_csv:76 - Entity 5d50a881-c383-463e-8355-41b3dd57422d not in input entities


In [ ]:
# NBVAL_SKIP
from pypsdm.plots.grid import grid_plot

# Use the grid_plot method to visualize the grid model
# only works if the underlying node input files have associated coordinates
grid_plot(gwr.grid)

In [3]:
results = gwr.results
# The grid results are symmetrical to the input grid, so there is a result container
# for participants and for the raw grid.
raw_grid_res = results.raw_grid
participants_res = results.participants

In [4]:
# Results for each grid element and participant is a discrete event time series
# which basically means each recorded state of e.g. a node (consisting of current magnitude, and angle)
# is valid until the next recorded state.

nodes_res = raw_grid_res.nodes
# Reminder: you can also access the nodes result directly from the gwr e.g. gwr.nodes_res
nodes_res = gwr.nodes_res

In [5]:
# The underlying structure of the result data is a dictionary subclass
# the values of the dictionary are dependent on the result model
# e.g. the result dictionary NodesResult contains ComplexVoltage objects


# You can access items as you would with a standard dictionary
uuid = list(nodes_res.keys())[0]
node_res = nodes_res[uuid]

# Similar to the input models the underlying data structure of each singular result object
# is a pandas DataFrame with a date time index
node_res.data.head()

,v_ang,v_mag
time,,
2016-01-02 00:00:00,-1.870384,1.028498
2016-01-02 01:00:00,-1.762598,1.029275
2016-01-02 02:00:00,-1.484783,1.031397
2016-01-02 03:00:00,-1.761274,1.029855
2016-01-02 04:00:00,-1.776910,1.029577


In [6]:
from datetime import datetime

# If you want to look at some time interval, you can slice the result models with date
# time objects
start = datetime(2016, 1, 1)
end = datetime(2016, 1, 1, 1)
sliced_node_res = node_res[start:end]
sliced_node_res.data

,v_mag,v_ang


In [7]:
# You can also filter the whole dictionary for a time interval
sliced_nodes_res = nodes_res.interval(start, end)
sliced_nodes_res[uuid].data

,v_mag,v_ang


In [8]:
# For participants all things look mostly the same
# The type of results are mostly of type PQResult, containing active and reactive power


loads_res = participants_res.loads
load_res = loads_res[list(loads_res.keys())[0]]
load_res.data.head()

,p,q
time,,
2016-01-02 00:00:00,0.062105,0.019104
2016-01-02 01:00:00,0.061128,0.018259
2016-01-02 02:00:00,0.163112,0.052141
2016-01-02 03:00:00,0.122642,0.038348
2016-01-02 04:00:00,0.169063,0.056043


In [9]:
# There are calculation utilities implemented where it makes sense
load_res2 = loads_res[list(loads_res.keys())[1]]
load_sum = load_res + load_res2
load_sum.data.head()

,p,q
time,,
2016-01-02 00:00:00,0.093124,0.033244
2016-01-02 01:00:00,0.084232,0.029093
2016-01-02 02:00:00,0.186671,0.062597
2016-01-02 03:00:00,0.149893,0.051081
2016-01-02 04:00:00,0.202280,0.071568


In [10]:
# You can also calculate the sum for all participants
total_loads_res = loads_res.sum()
total_loads_res.data.head()

,p,q
time,,
2016-01-02 00:00:00,10.749625,3.744239
2016-01-02 01:00:00,9.165363,3.183876
2016-01-02 02:00:00,10.923088,3.587497
2016-01-02 03:00:00,10.960480,3.691865
2016-01-02 04:00:00,12.256203,4.184011


In [11]:
# NBVAL_IGNORE_OUTPUT
# If you care about a specific attribute of all entities in a result dict you can assemble
# a data frame with all time series of all entities in the dict
loads_res_p = loads_res.p().head()
loads_res_p

,MV3.101 Load 35,MV3.101 Load 128,MV3.101 Load 32,MV3.101 Load 4,MV3.101 Load 102,MV3.101 Load 9,MV3.101 Load 127,MV3.101 Load 39,MV3.101 Load 30,MV3.101 Load 57,...,MV3.101 Load 124,MV3.101 MV Load 4,MV3.101 Load 41,MV3.101 Load 56,MV3.101 Load 22,MV3.101 Load 75,MV3.101 Load 83,MV3.101 Load 126,MV3.101 Load 132,MV3.101 Load 99
time,,,,,,,,,,,,,,,,,,,,,
2016-01-02 00:00:00,0.062105,0.031019,0.059087,0.059715,0.121197,0.043861,0.166768,0.005233,0.002842,0.042601,...,0.062863,0.009084,0.038872,0.096683,0.006709,0.076917,0.036702,0.033934,0.130073,0.092343
2016-01-02 01:00:00,0.061128,0.023104,0.028263,0.025887,0.100505,0.020536,0.148216,0.004820,0.002067,0.028449,...,0.043455,0.008364,0.039517,0.122517,0.007095,0.022330,0.032585,0.027471,0.116319,0.088498
2016-01-02 02:00:00,0.163112,0.023559,0.052578,0.089279,0.140857,0.022141,0.136638,0.011962,0.001951,0.048417,...,0.038899,0.007680,0.099994,0.209310,0.004764,0.080061,0.072743,0.026022,0.147758,0.080287
2016-01-02 03:00:00,0.122642,0.027250,0.048580,0.087788,0.149137,0.019354,0.153837,0.003419,0.001755,0.046959,...,0.042460,0.008035,0.072749,0.139747,0.004254,0.096344,0.057790,0.024603,0.113026,0.085756
2016-01-02 04:00:00,0.169063,0.033218,0.096387,0.093685,0.150302,0.030949,0.148948,0.014216,0.003455,0.108787,...,0.079728,0.008187,0.088585,0.139447,0.007080,0.092410,0.054426,0.023448,0.113710,0.104972


In [12]:
# NBVAL_IGNORE_OUTPUT
# By default, the data frame uses the ids of the loads if available instead of the uuids.
# You can change this behavior with the favor_ids argument
loads_res_p = loads_res.p(favor_ids=False).head()
loads_res_p

,0014e974-d490-4a1c-96e2-23d0e523b104,02a26927-da09-410f-8452-903177ef645b,05bdcb09-d1b4-4a30-b06a-b26d543d6929,07ff3b74-bda4-4a30-9584-3daa033068bb,095d4836-9075-49c1-bb2b-447db22fcb22,0acbb234-af80-44b3-8c5a-cff35e3bd7ac,0df30eab-565f-4184-8404-c33f80a3d1a6,0e64c305-f846-4bd4-8898-fbd833469bde,0ec9fd06-f5db-423c-96f1-03ddb88b9491,1199d1f7-22ae-47ad-91b4-e89c9f8fd16f,...,eecfdcb5-b327-48e3-a525-832d0209e7b8,f06c94f9-5ee5-4e60-bc20-3a2f62699f4d,f0b13ceb-da8a-4767-a441-d36980c15b72,f2447245-8a45-4980-8498-cf471d886365,f251d2a7-f131-4614-b906-418fa5750765,f2704836-9871-4f0d-8e62-433f2c95ede4,f2840dba-9c0c-48e9-b5b0-a25499ae51dd,f4d3b235-5403-4188-88a1-94b80dfa53b8,f6204e43-1cfd-4cd8-91b2-3846a980154f,ff389df3-f90d-41ef-b967-d92c0a51e918
time,,,,,,,,,,,,,,,,,,,,,
2016-01-02 00:00:00,0.062105,0.031019,0.059087,0.059715,0.121197,0.043861,0.166768,0.005233,0.002842,0.042601,...,0.062863,0.009084,0.038872,0.096683,0.006709,0.076917,0.036702,0.033934,0.130073,0.092343
2016-01-02 01:00:00,0.061128,0.023104,0.028263,0.025887,0.100505,0.020536,0.148216,0.004820,0.002067,0.028449,...,0.043455,0.008364,0.039517,0.122517,0.007095,0.022330,0.032585,0.027471,0.116319,0.088498
2016-01-02 02:00:00,0.163112,0.023559,0.052578,0.089279,0.140857,0.022141,0.136638,0.011962,0.001951,0.048417,...,0.038899,0.007680,0.099994,0.209310,0.004764,0.080061,0.072743,0.026022,0.147758,0.080287
2016-01-02 03:00:00,0.122642,0.027250,0.048580,0.087788,0.149137,0.019354,0.153837,0.003419,0.001755,0.046959,...,0.042460,0.008035,0.072749,0.139747,0.004254,0.096344,0.057790,0.024603,0.113026,0.085756
2016-01-02 04:00:00,0.169063,0.033218,0.096387,0.093685,0.150302,0.030949,0.148948,0.014216,0.003455,0.108787,...,0.079728,0.008187,0.088585,0.139447,0.007080,0.092410,0.054426,0.023448,0.113710,0.104972


## Nodal Results

Often times we care about nodal results, so the net load and generation at individual
nodes. Therefore, we include a special data model `ExtendedNodesResult` which is
calculated by summing up the results of all connected participants at the individual nodes.

In [13]:
# There is a method to calculate all of them
nodal_results = gwr.build_extended_nodes_result()

target_uuid = list(nodal_results.keys())[3]
nodal_results[target_uuid].data.head()

,v_ang,v_mag,p,q
time,,,,
2016-01-02 00:00:00,-2.131056,1.020498,0.223068,0.099645
2016-01-02 01:00:00,-2.008640,1.021802,0.199644,0.090745
2016-01-02 02:00:00,-1.697719,1.024779,0.184939,0.085148
2016-01-02 03:00:00,-2.042256,1.021914,0.207742,0.092404
2016-01-02 04:00:00,-2.044741,1.021885,0.195353,0.088521


In [14]:
# If you care about the individual systems at each node, you can filter down the
# result container to the systems connected to a specified node

nodal_gwr = gwr.nodal_result(target_uuid)
nodal_gwr.loads.p()

,MV3.101 Load 61
time,
2016-01-02 00:00:00,0.155059
2016-01-02 01:00:00,0.150625
2016-01-02 02:00:00,0.147085
2016-01-02 03:00:00,0.148944
2016-01-02 04:00:00,0.158602
...,...
2016-01-07 20:00:00,0.180501
2016-01-07 21:00:00,0.182086
2016-01-07 22:00:00,0.165844


In [15]:
# NBVAL_IGNORE_OUTPUT
# Let's say you want to make sure that all the load is actually connected to the
# node that we have filtered for
load_uuids = list(nodal_gwr.loads.keys())

# You can get a subset of the input data model via a list of uuids
# Note that the `node` attribute confirms that indeed the load is connected
# to the node we have filtered for
gwr.loads.subset(load_uuids[0]).data

,cos_phi_rated,e_cons_annual,id,load_profile,node,operates_from,operates_until,operator,q_characteristics,s_rated
uuid,,,,,,,,,,
a0330517-9705-4d0a-bcaf-71f203cd6187,0.93,0.0,MV3.101 Load 61,No load profile assigned,090d13e8-3cce-4793-816f-4c50f23f3f7f,NaN,NaN,NaN,"cosPhiFixed:{(0.0,0.93)}",668.0


For more details please check the implemented methods of all the data models

Check out the `docs/nbs/plots.ipynb` notebook for some examples of the included plotting utilities


## Line Results
# Here is some example for line results to analyse for line ratings etc.

In [22]:
# If you care about the individual line element, you can filter down the
# result container to the systems connected to a specified line
line_uuid = gwr.lines.data.iloc[0].name
line_gwr = gwr.lines_res[line_uuid]
line_gwr.data.head()

,i_a_ang,i_a_mag,i_b_ang,i_b_mag
time,,,,
2016-01-02 00:00:00,-16.907664,92.385576,162.980422,92.433464
2016-01-02 01:00:00,-16.605600,85.653368,163.273446,85.700759
2016-01-02 02:00:00,-16.931381,73.715509,162.928117,73.765059
2016-01-02 03:00:00,-14.665853,93.995163,165.222982,94.036282
2016-01-02 04:00:00,-14.757302,92.907599,165.130276,92.949001


In [27]:
# NBVAL_IGNORE_OUTPUT
# If we would like to get the utilisation of the line segment, we need use i_max provide from input data (gwr.lines)

line_input_data = gwr.lines
line_utilization = gwr.lines_res.utilisation(line_input_data, side="a")
line_utilization.head()

,00272097-2670-45dd-a400-ef1fd7531b71,01137d2a-a5e5-417c-93f1-19c08b9f2782,02a3b3ed-a10e-422a-9f92-7bf3b84cf619,04971185-dac9-4e22-85dc-bae8f112b7a3,07cd4ac4-2b74-400f-b5f5-04bf9b3cffd2,0d01725a-e0f6-4a1e-ba57-a5888c088606,0da2a417-5bbf-4360-a3d9-1d566ab538b3,11ffde44-16bc-4826-860e-f65f95e4fd35,1acb110b-0cb6-42f5-8099-52aff310ec86,1af614d9-62b8-40e2-be73-c69509181251,...,f0543101-82d2-4a5d-be4d-5e3eec20df92,f20eb95b-b5e6-450d-b08f-0564d1407596,f3513e99-6ee1-4320-bedb-95a60d39a29f,f51207e1-4c3c-4018-81da-c028d0f20224,f534791f-e9cf-4094-b272-a4f2f2547749,f7f31262-bb20-4aee-8acb-f882136e8436,fb47eb8f-ecd4-4631-a086-2243a13d90ab,fce26ed8-d2a0-495f-9bfa-8a1f582fc070,fedf69d6-ed13-4043-b097-7881caffc82a,ff1fe640-dc63-4ee7-ae88-38a2c0ca4430
time,,,,,,,,,,,,,,,,,,,,,
2016-01-02 00:00:00,0.000591,0.044685,0.163577,0.082521,0.028229,0.278192,0.055601,0.084160,0.152561,0.152327,...,0.168149,0.001327,0.246417,0.068733,0.190255,0.106950,0.103283,0.043909,0.139741,0.275812
2016-01-02 01:00:00,0.000592,0.039060,0.148156,0.073057,0.025225,0.264976,0.052485,0.083692,0.136333,0.143703,...,0.160983,0.001329,0.236243,0.065432,0.173475,0.093307,0.090149,0.042703,0.127455,0.264586
2016-01-02 02:00:00,0.000593,0.032822,0.133065,0.057157,0.020361,0.232802,0.045257,0.067240,0.123094,0.111405,...,0.132443,0.001331,0.207355,0.056214,0.154330,0.077204,0.067478,0.037128,0.108268,0.231060
2016-01-02 03:00:00,0.000592,0.038667,0.158997,0.069731,0.018309,0.248936,0.048743,0.076725,0.140345,0.132976,...,0.147030,0.001329,0.220347,0.056984,0.182296,0.081870,0.088845,0.041619,0.124198,0.249711
2016-01-02 04:00:00,0.000592,0.036513,0.160186,0.081915,0.018864,0.259945,0.044201,0.075470,0.144465,0.137517,...,0.144734,0.001329,0.231201,0.063116,0.177682,0.083896,0.104677,0.043273,0.121328,0.263668


In [24]:
line_utilization[[line_uuid]].head()

,26bdf38b-c420-4d3e-9b08-486541d2dd2d
time,
2016-01-02 00:00:00,0.196148
2016-01-02 01:00:00,0.181854
2016-01-02 02:00:00,0.156509
2016-01-02 03:00:00,0.199565
2016-01-02 04:00:00,0.197256


## Congestion Results

If we want to analyze congestions in the grid, we can run SIMONA with congestion detection. All congestion results are mapped uuid of the asset for which the congestion was detected. The type information (e.g.: node, line, ect.) specifies type of the asset that has a congestion.

Each congestion contains the value that occurred and the limits for the asset (e.g.: voltage band, line current limit, etc.).

In [ ]:
# Congestion results are only provided for two nodes
node_with_congestions = [
    "5d50a881-c383-463e-8355-41b3dd57422d",
    "557b9f51-d83c-476c-a84c-d240530c203d",
]

congestion_res_1 = gwr.congestions_res[node_with_congestions[0]]
congestion_res_2 = gwr.congestions_res[node_with_congestions[1]]

congestion_res_1.data